<a href="https://colab.research.google.com/github/MuhammadIrzam447/visionCodes/blob/master/Debuggingv4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Testing Multi-model performance for 100% images and _% text.
# Updated Validation Dataset Class and Validation Loop

In [1]:
from google.colab import drive
import pandas as pd
import os
from PIL import Image
import torch
from torchvision import datasets, transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
import torchvision
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import UnidentifiedImageError
from sklearn.metrics import classification_report
# import patoolib

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cuda


# Loading Validation Dataset and Preprocessing

In [ ]:
save_dir = '/content/Model/ResNet_Joint_Representation'
load_path = os.path.join(save_dir, 'model.pth')

# Create an instance of the ResNet model
resnet = torchvision.models.resnet101(pretrained=False)
resnet.fc = nn.Linear(2048, 52)

# Load the saved model parameters
resnet.load_state_dict(torch.load(load_path))

# Set the model to evaluation mode and respective device
resnet.eval()
resnet.to(device)

In [4]:
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
class ValidationDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, val_transform):
        self.data_dir = data_dir
        self.dataset = datasets.ImageFolder(data_dir)
        self.classes = self.dataset.classes
        self.class_lengths = self._compute_class_lengths()
        self.val_transform = val_transform

        # Load only 50% of data from each class
        self.selected_indices = []
        for class_idx in range(len(self.classes)):
            indices = [idx for idx, (_, label) in enumerate(self.dataset.samples) if label == class_idx]
            indices_3 = [idx for idx in indices if self.dataset.samples[idx][0].endswith("_3.png")]    # _3.png are the encoded_text images
            indices_4 = [idx for idx in indices if self.dataset.samples[idx][0].endswith("_4.png")]    # _4.png are the actual images

            # Include all files ending with "_4.png"
            self.selected_indices.extend(indices_4)

            # Randomly select 50% of files ending with "_3.png"
            num_samples_3 = len(indices_3)
            num_samples_to_load_3 = int(1 * num_samples_3) # change the value to change percentage of data <<<<<<<<<<<<<
            # selected_indices_3 = np.random.choice(indices_3, num_samples_to_load_3, replace=False)

            # Append the selected indices to the selected_indices list
            self.selected_indices.extend(indices_3)

        print("Selected Indices:", len(self.selected_indices))
        # for idx in self.selected_indices:
          # print(self.dataset.samples[idx][0])

    def _compute_class_lengths(self):
        class_lengths = {cls: 0 for cls in self.classes}

        for _, label in self.dataset.samples:
            class_lengths[self.classes[label]] += 1

        return class_lengths

    def __getitem__(self, index):
        print("Entered get_item")
        img, label = self.dataset[self.selected_indices[index]]
        filename = self.dataset.samples[self.selected_indices[index]][0]
        image_3 = None
        image_4 = None
        if filename.endswith("_4.png"):
                image_4 = img
                image_4_path = filename
                print("image_4_path: ", str(filename))
                image_3_path = filename.replace('_4.png', '_3.png')
                print("image_3_path: ", str(image_3_path))
                image_3 = self._load_image(image_3_path)
        else:
                image_3 = img
                image_3_path = filename
                print("image_3_path: ", str(filename))
                image_4_path = filename.replace('_3.png', '_4.png')
                print("image_4_path: ", str(image_4_path))
                image_4 = self._load_image(image_4_path)


        return image_3, image_4, label, image_3_path, image_4_path

    def _load_image(self, path):
        image = Image.open(path)
        return image

    def __len__(self):
        return len(self.selected_indices)

In [6]:
valPath = "/content/Data/mulitmodal_img_enc_txt/images-val"
val_dataset = ValidationDataset(valPath,val_transform)

Selected Indices: 43738


In [7]:
from torchvision.transforms import ToTensor

def custom_collate(batch):
    to_tensor = ToTensor()

    images_3 = [to_tensor(item[0]) for item in batch]
    images_4 = [to_tensor(item[1]) for item in batch]
    labels = [torch.tensor(item[2]) for item in batch]  # Convert labels to tensors

    # Filter out None values from images_4
    # images_4 = [img for img in images_4 if img is not None]

    # Convert the list of images to tensors
    images_3 = torch.stack(images_3) if images_3 else None
    images_4 = torch.stack(images_4)

    # Print the filenames in each list
    print("Filenames in images_3 list:")
    for item in batch:
        if item[0] is not None:
            print(item[3])

    print("Filenames in images_4 list:")
    for item in batch:
        if item[1] is not None:
            print(item[4])

    return images_3, images_4, labels



In [8]:
batch_size = 12
validation_data_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate)

In [9]:
print("Number of samples:", len(val_dataset))
print("Number of classes:", len(val_dataset.classes))

Number of samples: 43738
Number of classes: 52


# Validation

In [10]:
resnet.eval()

# Initialize lists to store the average probabilities and true labels
predicted_labels = []
true_labels = []

# Iterate over the dataloader in your testing loop
for images_3, images_4, labels in validation_data_loader:
    # Move the images and labels to the device (e.g., GPU) if available
    images_3 = images_3.to(device)
    images_4 = images_4.to(device)

    # logits_4 = resnet(images_4)
    logits_3 = resnet(images_3)
    logits_4 = resnet(images_4)
    # logits_4 = torch.zeros_like(logits_3)

    # Compute the probabilities using softmax along the class dimension
    probabilities_3 = torch.softmax(logits_3, dim=1)
    probabilities_4 = torch.softmax(logits_4, dim=1)

    # Average the probabilities for each pair
    avg_probabilities_batch = (probabilities_3 + probabilities_4) / 2

    # Append the average probabilities to the list
    predicted_labels.extend(avg_probabilities_batch.cpu().tolist())

    # labels = torch.tensor(labels).to(device)
    # Append the true labels to the list
    true_labels.extend(labels)


Streaming output truncated to the last 5000 lines.
Entered get_item
image_3_path:  /content/Data/mulitmodal_img_enc_txt/images-val/@vite_bullone/175474067.jpg_3.png
image_4_path:  /content/Data/mulitmodal_img_enc_txt/images-val/@vite_bullone/175474067.jpg_4.png
Entered get_item
image_3_path:  /content/Data/mulitmodal_img_enc_txt/images-val/@vite_bullone/175474082.jpg_3.png
image_4_path:  /content/Data/mulitmodal_img_enc_txt/images-val/@vite_bullone/175474082.jpg_4.png
Entered get_item
image_3_path:  /content/Data/mulitmodal_img_enc_txt/images-val/@vite_bullone/175474193.jpg_3.png
image_4_path:  /content/Data/mulitmodal_img_enc_txt/images-val/@vite_bullone/175474193.jpg_4.png
Entered get_item
image_3_path:  /content/Data/mulitmodal_img_enc_txt/images-val/@vite_bullone/175474900.jpg_3.png
image_4_path:  /content/Data/mulitmodal_img_enc_txt/images-val/@vite_bullone/175474900.jpg_4.png
Entered get_item
image_3_path:  /content/Data/mulitmodal_img_enc_txt/images-val/@vite_bullone/175475083.j

In [11]:
predicted_classes = torch.argmax(torch.tensor(predicted_labels), dim=1)
actual_labels = torch.tensor(true_labels)

In [12]:
accuracy = accuracy_score(actual_labels, predicted_classes)
precision = precision_score(actual_labels, predicted_classes, average='weighted')
recall = recall_score(actual_labels, predicted_classes, average='weighted')
f1 = f1_score(actual_labels, predicted_classes, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.48113768347889707
Precision: 0.9077952689068244
Recall: 0.48113768347889707
F1-score: 0.5638646255376258


In [ ]:
print(classification_report(actual_labels, predicted_classes))

In [ ]:
labels = ['Accuracy', 'Precision', 'Recall', 'F1-score']
values = [accuracy, precision, recall, f1]

x = np.arange(len(labels))

fig, ax = plt.subplots(figsize=(5, 4))
bars = ax.bar(x, values)

# labels, title, and legend
ax.set_xlabel('Metrics')
ax.set_ylabel('Score')
ax.set_title('Model Performance Metrics')
ax.set_xticks(x)
ax.set_xticklabels(labels)

# scores on top of each bar
for i, bar in enumerate(bars):
    score = values[i]
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{score:.2f}', ha='center', va='bottom')

plt.show()

In [ ]:
# Count the occurrences of each label in predicted and true labels
predicted_counts = np.bincount(predicted_classes)
true_counts = np.bincount(actual_labels)

# Get the unique labels
labels = np.unique(np.concatenate((predicted_classes, actual_labels)))

# Set the x-axis range
x = np.arange(len(labels))

# Set the width of the bars
width = 0.35

# Plot the predicted and true label counts
fig, ax = plt.subplots(figsize=(20, 8))
ax.bar(x - width/2, predicted_counts, width, label='Predicted Labels')
ax.bar(x + width/2, true_counts, width, label='True Labels')

# Add labels, title, and legend
ax.set_xlabel('Labels')
ax.set_ylabel('Count')
ax.set_title('Distribution of Predicted and True Labels')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

plt.show()